In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/genius-song-lyrics-with-language-information/song_lyrics.csv


In [3]:
os.environ["UNSLOTH_DISABLE_CUSTOM_LOSS"] = "1"
os.environ["UNSLOTH_DISABLE_TRITON"] = "1"
os.environ["TRITON_CACHE_DIR"] = "/tmp/triton_cache"
os.environ["CCL_DISABLE_TRITON"] = "1"
os.environ["DISABLE_UNSLOTH_TRITON"] = "1"

In [4]:
%pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 1.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.6/218.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 60.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 MB 2.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 4.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 107.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 80.2 MB/s eta 0:00:0

In [5]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
import os
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-05-02 10:13:38.441687: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746180818.639203      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746180818.693504      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


In [6]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "song_lyrics.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "carlosgdcj/genius-song-lyrics-with-language-information",
  file_path,
  # Provide any additional arguments like 
  # sql_query or pandas_kwargs. See the 
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", df.head())

/tmp/ipykernel_31/1288909252.py:10: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  df = kagglehub.load_dataset(


First 5 records:                title  tag     artist  year   views  \
0          Killa Cam  rap    Cam'ron  2004  173166   
1         Can I Live  rap      JAY-Z  1996  468624   
2  Forgive Me Father  rap   Fabolous  2003    4743   
3       Down and Out  rap    Cam'ron  2004  144404   
4             Fly In  rap  Lil Wayne  2005   78271   

                                       features  \
0                   {"Cam\\'ron","Opera Steve"}   
1                                            {}   
2                                            {}   
3  {"Cam\\'ron","Kanye West","Syleena Johnson"}   
4                                            {}   

                                              lyrics  id language_cld3  \
0  [Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki...   1            en   
1  [Produced by Irv Gotti]\n\n[Intro]\nYeah, hah,...   3            en   
2  Maybe cause I'm eatin\nAnd these bastards fien...   4            en   
3  [Produced by Kanye West and Brian Miller]\n\n[...   

In [7]:
df = df.groupby('artist').filter(lambda x: len(x) >= 2)
df = df.groupby('artist').apply(lambda x: x.sample(n=2, random_state=42)).reset_index(drop=True)


/tmp/ipykernel_31/3358028032.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('artist').apply(lambda x: x.sample(n=2, random_state=42)).reset_index(drop=True)


In [8]:
if len(df) > 650000:
    df = df.sample(n=650000, random_state=42)

In [9]:
df.drop(columns = ["tag","year","views","features","id","language_cld3","language_ft","language"],inplace=True)

In [10]:
df['instruction'] = "Generate a song in the style of " + df['artist']
df['response'] = "Title: " + df['title'] + "\nLyrics:\n" + df['lyrics']


In [11]:
chat_template = """Below are some instructions that describe some tasks. Write responses that appropriately complete each request.

### Instruction:
{INPUT}

### Response:
{OUTPUT}"""

In [12]:
df['text'] = df.apply(lambda row: chat_template.format(INPUT=row['instruction'], OUTPUT=row['response']), axis=1)


In [13]:
train_df, eval_df = train_test_split(df, test_size=0.2, random_state=42)


In [14]:
train_dataset = Dataset.from_pandas(train_df[['text']])
eval_dataset = Dataset.from_pandas(eval_df[['text']])

In [15]:
train_dataset[0]

{'text': 'Below are some instructions that describe some tasks. Write responses that appropriately complete each request.\n\n### Instruction:\nGenerate a song in the style of Luke Goliath x Lynxmack\n\n### Response:\nTitle: Nothing\nLyrics:\nYou don’t gotta hate on me\nYou don’t gotta say nothing\nGirl I know that you love\u2005me\u2005(that you love\u2005me)\n(Love, me)\nI mean it’s easy to\u2005see (to see, to see)\nThat don’t got no filter (that you don’t got no filter)\n\nI been focused on you\nI been rolling for you (for you)\nSmoking for you (for you)\nAnd baby don’t waste no time get straight to the point (straight to the point)\nYeah\nI know you don’t got no filter (you don’t got no filter)\n\nYou just make me feel some way\nYou just make, you just make me wanna come down tonight\nAnd I don’t care if your nigga home tonight (if your nigga home tonight)\nI was hoping you alone tonight\nTryna get this work tonight\n\nSunrise to sundown, yeah your eyes in the light tonight (tonigh

In [16]:
compute_dtype = torch.bfloat16 if is_bfloat16_supported() else torch.float16
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit",
    max_seq_length=2048,
    dtype=compute_dtype,
    load_in_4bit=True,
)

==((====))==  Unsloth 2025.4.4: Fast Llama patching. Transformers: 4.51.1.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 6.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [17]:
model = FastLanguageModel.get_peft_model(
    model,
    r=4,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
    use_rslora=False,
    loftq_config=None,
)


Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.4.4 patched 32 layers with 32 QKV layers, 32 O layers and 0 MLP layers.


In [18]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=4, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=4, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4b

In [19]:
os.environ["UNSLOTH_RETURN_LOGITS"] = "1"

In [20]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Configuration specifically optimized for Tesla T4
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 1,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 15,
        learning_rate = 2e-4,
        # T4 supports FP16 but not BF16
        fp16 = True,     # Enable FP16 which is supported
        bf16 = False,    # Explicitly disable BF16 which is not supported
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)


Unsloth: Tokenizing ["text"]:   0%|          | 0/520000 [00:00<?, ? examples/s]

In [21]:
trainer_stats = trainer.train()


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 520,000 | Num Epochs = 1 | Total steps = 15
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 3,407,872/8,000,000,000 (0.04% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.170400
2,3.098900
3,2.643300
4,2.519200
5,2.777300
6,2.713800
7,2.867400
8,3.196700
9,2.756000
10,3.200900


In [23]:
# Save model and tokenizer
model.save_pretrained("/kaggle/working/my_model")
tokenizer.save_pretrained("/kaggle/working/my_model")

# Compress the directory
import shutil
shutil.make_archive("my_model", 'zip', "/kaggle/working/my_model")

'/kaggle/working/my_model.zip'

In [24]:
from IPython.display import FileLink
display(FileLink(r'my_model.zip'))

/kaggle/working/my_model.zip